In [11]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')

data_path = 'D:\\Source\\ML100Days_Data\\D4\\'
df = pd.read_csv(data_path + 'titanic_train.csv')

train_Y = df['Survived']
df = df.drop(['PassengerId', 'Survived'] , axis=1)
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [12]:
# 取一個類別型欄位, 與一個數值型欄位, 做群聚編碼
# 年齡(Age)對艙等(Pclass)做群聚編碼
"""
Your Code Here
"""
df['Age'] = df['Age'].fillna(df['Age'].mean())
df['Sex'] = LabelEncoder().fit_transform(df['Sex']).astype('int64')
mean_df = df.groupby(['Sex'])['Age'].mean().astype('float64').reset_index()
mode_df = df.groupby(['Sex'])['Age'].apply(lambda x: x.mode()[0]).reset_index()
median_df = df.groupby(['Sex'])['Age'].median().reset_index()
max_df = df.groupby(['Sex'])['Age'].max().reset_index()
min_df = df.groupby(['Sex'])['Age'].min().reset_index()
temp = pd.merge(mean_df, mode_df, how='left', on=['Sex'])
temp = pd.merge(temp, median_df, how='left', on=['Sex'])
temp = pd.merge(temp, max_df, how='left', on=['Sex'])
temp = pd.merge(temp, min_df, how='left', on=['Sex'])
temp.columns = ['Sex', 'Age_Mean', 'Age_Mode', 'Age_Median', 'Age_Max','Age_Min']
temp

,Sex,Age_Mean,Age_Mode,Age_Median,Age_Max,Age_Min
0,0,28.216730,29.699118,29.699118,63.0,0.75
1,1,30.505824,29.699118,29.699118,80.0,0.42


In [13]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
df = df.fillna(-1)
MMEncoder = MinMaxScaler()
df.head()

6 Numeric Features : ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare']



,Pclass,Sex,Age,SibSp,Parch,Fare
0,3,1,22.0,1,0,7.2500
1,1,0,38.0,1,0,71.2833
2,3,0,26.0,0,0,7.9250
3,1,0,35.0,1,0,53.1000
4,3,1,35.0,0,0,8.0500


In [14]:
# 原始特徵 + 邏輯斯迴歸
"""
Your Code Here
"""
train_num = train_Y.shape[0]
train_X = df[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
df.head()

0.7912663261444924


,Pclass,Sex,Age,SibSp,Parch,Fare
0,3,1,22.0,1,0,7.2500
1,1,0,38.0,1,0,71.2833
2,3,0,26.0,0,0,7.9250
3,1,0,35.0,1,0,53.1000
4,3,1,35.0,0,0,8.0500


In [15]:
df = pd.merge(df, temp, how='left', on=['Sex'])
#df = df.drop(['Pclass'] , axis=1)
df.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Age_Mean,Age_Mode,Age_Median,Age_Max,Age_Min
0,3,1,22.0,1,0,7.2500,30.505824,29.699118,29.699118,80.0,0.42
1,1,0,38.0,1,0,71.2833,28.216730,29.699118,29.699118,63.0,0.75
2,3,0,26.0,0,0,7.9250,28.216730,29.699118,29.699118,63.0,0.75
3,1,0,35.0,1,0,53.1000,28.216730,29.699118,29.699118,63.0,0.75
4,3,1,35.0,0,0,8.0500,30.505824,29.699118,29.699118,80.0,0.42


In [16]:
# 新特徵 + 邏輯斯迴歸
"""
Your Code Here
"""
train_X = df[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
df.head()

0.7868034003986825


,Pclass,Sex,Age,SibSp,Parch,Fare,Age_Mean,Age_Mode,Age_Median,Age_Max,Age_Min
0,3,1,22.0,1,0,7.2500,30.505824,29.699118,29.699118,80.0,0.42
1,1,0,38.0,1,0,71.2833,28.216730,29.699118,29.699118,63.0,0.75
2,3,0,26.0,0,0,7.9250,28.216730,29.699118,29.699118,63.0,0.75
3,1,0,35.0,1,0,53.1000,28.216730,29.699118,29.699118,63.0,0.75
4,3,1,35.0,0,0,8.0500,30.505824,29.699118,29.699118,80.0,0.42
